# Cifar-10 testset classification on Pynq

This notebook covers how to use low quantized neural networks on Pynq. 
It shows an example how CIFAR-10 testset can be inferred utilizing a convolutional neural network featuring 3 convolutional layers, 2 max pool layers and 3 fully connected layers. There is 1 precision available:

- CNV_h1_W1A1 using 1 bit weights and 1 bit activation,

## 1. Import the package

In [1]:
import bnn

## 2. The Cifar-10 testset

This notebook required the testset from https://www.cs.toronto.edu/~kriz/cifar.html which contains 10000 images that can be processed by CNV network directly without preprocessing.

You can download the cifar-10 set from given url via wget and unzip it to a folder on Pynq as shown below.
This may take a while as the training set is included in the archive as well.
After that we need to read the labels from the binary file to be able to compare the results later:

In [3]:
#get
!wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
#unzip
!tar -xf cifar-10-binary.tar.gz

labels = []
with open("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/test_batch.bin", "rb") as file:
    #for 10000 pictures
    for i in range(10000):
        #read first byte -> label
        labels.append(int.from_bytes(file.read(1), byteorder="big"))
        #read image (3072 bytes) and do nothing with it
        file.read(3072)
    file.close()

--2020-04-12 13:17:05--  https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170052171 (162M) [application/x-gzip]
Saving to: ‘cifar-10-binary.tar.gz’

cifar-10-binary.tar 100%[===================>] 162.17M  8.00MB/s    in 19s     

2020-04-12 13:17:25 (8.59 MB/s) - ‘cifar-10-binary.tar.gz’ saved [170052171/170052171]



## 3. Start inference

The inference can be performed with different precision for weights and activation. Creating a specific Classifier will automatically download the correct bitstream onto PL and load the weights and thresholds trained on the specific dataset. 

Thus that images are already Cifar-10 preformatted no preprocessing is required. Therefor the functions `classify_cifar` or `classify_cifars` can be used. When classifying non Cifar-10 formatted pictures refer to `classify_image` or `classify_images`  (see Notebook CNV-QNN_Cifar10).

### Case 1: 
#### W1A1 - 1 bit weight and 1 activation

Instantiate the classifier:

In [4]:
hw_classifier = bnn.CnvClassifier(bnn.NETWORK_HCNVW1A1,'cifar10',bnn.RUNTIME_HW)

[DEBUG] Datasets: ['chars_merged', 'streetview', 'cifar10', 'road-signs', 'mnist']
[DEBUG] ret: ['cifar10']
[DEBUG] Initializing PynqBNN class . . . 
	runtime: python_hw
	network: hcnvW1A1
[DEBUG] Overlay Loaded
[DEBUG] Loading python_hw-hcnvW1A1-Zybo-Z7.so . . . 
[DEBUG] PynqBNN class initialized.
[DEBUG] Sourcing params at: /usr/local/lib/python3.6/dist-packages/bnn/params/cifar10/hcnvW1A1


And start the inference on Cifar-10 preformatted multiple images:

In [5]:
result_W1A1 = hw_classifier.classify_cifars("/home/xilinx/jupyter_notebooks/bnn/cifar-10-batches-bin/test_batch.bin")
time_W1A1 = hw_classifier.usecPerImage

Inference took 3933922.12 microseconds, 393.39 usec per image
Classification rate: 2541.99 images per second


### Accuracy

The accuracy on the testset can be calculated by comparing the inferred labels against the one read at the beginning:


In [6]:
#compare against labels
countRight = 0
for idx in range(len(labels)):
    if labels[idx] == result_W1A1[idx]:
        countRight += 1
accuracyW1A1 = countRight*100/len(labels)

print("Accuracy W1A1: ",accuracyW1A1,"%")

Accuracy W1A1:  61.18 %


## 6. Reset the device

In [7]:
from pynq import Xlnk

xlnk = Xlnk()
xlnk.xlnk_reset()